# Project 3

The developing community of Dunetown has decided to use a large lake located in the highlands near the town as their water supply.  To accomplish this, they have decided to install a steel pipe to bring the water from the lake to the town, as shown in the figure below.  The pipe should be capable of delivering a flow rate of $10000$ gallons per minute to the town in order to satisfy their present demand for water.

You are an engineer in the Dunetown Department of Water Services and the department head has just charged you with determining the required diameter of the pipe.  She then hands you a list of tasks she would like to see completed in the analysis (See below).


<img src='images/dunetown.png' width=500>

## Analysis

Fully developed flow in a pipe is governed by an energy balance equation of the form

\begin{equation}
    \frac{p_1}{\rho} + \frac{V_{1}^2}{2} + g z_1 = \frac{p_2}{\rho} + \frac{V_{2}^2}{2} + g z_2 + g h_l,
\end{equation}

where station 1 is the free surface of the water in the lake and station 2 is the end of the pipe, $p$ is the pressure, $V$ is the average velocity of the fluid at that point, $z$ is the elevation, $\rho$ is the fluid density, $g$ is acceleration of gravity, and $h_l$ is the head loss, which represents irreversible transfer of mechanical energy to thermal energy in the flow.

In the figure above, the pressures at stations 1 and 2 are the same, the average velocity at the surface of the lake is zero, the elevation at $z_2$ is defined as zero, and the velocity at station 2 is the average velocity in the pipe $V$.  Thus the energy balance simplifies to

\begin{equation}
    z_1=\frac{V^2}{2 g}+h_l.
\end{equation}


The head loss is written as

\begin{equation}
    h_l=f \frac{L}{D} \frac{V^2}{2 g},
\end{equation}

where $f$ is the friction factor, $L$ is the pipe length, $D$ is the pipe diameter, and $V$ is the average velocity in the pipe.

The Colebrook friction factor for turbulent pipe flow is based on a correlation of a large amount of turbulent pipe flow data.  It is written as

\begin{equation}
    \frac{1}{\sqrt{f}} = -2.0 \log_{10} \left( \frac{\varepsilon/D}{3.7} + \frac{2.51}{Re \sqrt{f}} \right).
\end{equation}

Here, $\varepsilon$ is the pipe roughness and $Re$ is the Reynold's number of the flow, defined as

\begin{equation}
    Re =  \frac{VD}{\nu},
\end{equation}

where $\nu$ is the kinematic viscosity of the fluid. A good initial guess for this friction factor is

\begin{equation}
    f = \frac{1.325}{\left[ \ln \left( \frac{\varepsilon/D}{3.7} + \frac{5.74}{Re^{0.9}} \right) \right]^2}.
\end{equation}

The Colebrook friction factor for turbulent flow is valid for $Re > 2000$.  For $Re < 2000$, the flow is laminar and the friction factor is written simply as

\begin{equation}
    f = \frac{64}{Re}
\end{equation}

The flow rate $Q$ in the pipe is given by

\begin{equation}
    Q = A V,
\end{equation}

where $A$ is the cross sectional area of the pipe.

Finally, data for this pipe flow is given in the following table:

| Pipe Flow Data ||
|--:|:--|
| length, $L$ | 2 km |
| roughness, $\varepsilon$ | 0.2 mm |
| lake elevation, $z_1$ | 30 m |
| fluid viscosity, $\nu$ | $$1.12 \times 10^{-6} \mbox{m}^2/\mbox{s}$$ |
| acc. gravity, $g$ | 9.807 $\mbox{m}^2/\mbox{s}$ |

## Task 1

Complete the class `Dunetown` below by writing a member function `friction_factor()` to compute the friction factor for pipe flow.

 * The function should return a floating point number (not a NumPy array) that is the friction factor $f$.  The function takes two input arguments, the flow Reynold's number $Re$ and optionally, the pipe roughness ratio $\varepsilon / D$.  If the roughness ratio is omitted, use a default of zero, a smooth pipe.

 * Use the laminar friction factor when $Re < 2000$ and the Colebrook friction factor otherwise.

 * Use Newton's method to iterate the Colebrook formula.  Terminate the iteration when the approximate relative error is less than $10^8$.  Use the initial guess given to start your iteration.  With this starting point, your function should always converge.

 * If you think about it carefully, a change of variables makes your function for Newton's method easier to write.
 
 * You can write your own Newton's method implementation or use SciPy's [`scipy.optimize.fsolve`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.fsolve.html) function.  If you use `fsolve` be sure to include the optional argument `fprime` so that it uses Newton's method and not the Secant Method.
 
## Task 2

Write another member function called `pipeflow()` to solve the energy balance.

 * This function should have one argument, the pipe diameter $D$ in units of meters and return the average velocity in the pipe $V$ in meters per second.

 * Use any root finding method you choose to solve for the velocity in the energy balance.

 * Use a relative error of $10^{-8}$.

 * Test your function with a test diameter of $D=0.1$ m.  The average velocity for this case is $V=1.080244$ $\mbox{m}/\mbox{s}$.
 
 * The constants needed to carry out the calculations are already stored as class attributes.


## Task 3

Complete the member function `pipe_selector()`.  There is a class attribute called `pipe_sizes` that contains a dictionary of standard schedule 40 (SH40) pipe sizes.  The keywords in the dictionary are the "nominal" pipe sizes and the values are the actual inner diameter of the pipes (what we need to use to size the pipe for the flowrate).  Use the other functions you wrote and the `pipe_sizes` dictionary to determine the *smallest* nominal pipe size (an integer) that is adequate for the 10000 gal/min water demand.  `pipe_selector()` should return this pipe size.  **Note:** All of the other calculations should be accomplished in SI units.  We only convert to English units in this last step to size the pipe which are described and sold by their nominal sizes in inches.

In [4]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

class Dunetown(object):
    
    def __init__(self):   
        self.length = 2000        #m
        self.ϵ = 0.2 / 1000       #m
        self.z1 = 30              #m
        self.nu = 1.12e-6          #m^2/s
        self.gravity = 9.807      #m/s^2
        
        #key: value -> nominal SH40 pipe size in inches: actual pipe inner diameter in inches
        self.pipe_sizes = {4: 4.03, 5: 5.05, 6: 6.07, 8: 7.98, 10: 10.02, 
                           12: 11.94, 14: 13.13, 16: 15.00, 18: 16.88, 20: 18.81, 24: 22.63}
    
    def friction_factor(self, Re, roughness_ratio=0):
        if Re <= 2000:
            return 64/Re
        else:
            def colebrook(u):
                return u+2*np.log10(roughness_ratio/3.7+2.51/Re*u)
            def derivative(u):
                return 1+8.06659/(roughness_ratio*Re+9.287*u)
            initial_guess = 0.02
            f_solution = opt.fsolve(colebrook, initial_guess, fprime=derivative, xtol=1e-8)
            f_solution = f_solution**-2
        return f_solution[0]
    
        
    def pipeflow(self, diameter):
        def velocity_function(v):
            Re = v*diameter/self.nu
            f = self.friction_factor(Re, self.ϵ/diameter)
            hf = f*self.length/diameter*v**2/(2*self.gravity)
            return -self.z1+v**2/(2*self.gravity)+hf
        
        initial_guess = 1.0
        v_solution = opt.fsolve(velocity_function, initial_guess)
        return v_solution[0]
        
    def pipe_selector(self):
        flow_rate_gpm = 10000
        flow_rate_m3s = flow_rate_gpm * 0.00378541/60 
        
        adequate_pipe_size = None
        for nominal_size, diameter_inches in sorted(self.pipe_sizes.items()):
            diameter_m = diameter_inches * 0.0254
            area = np.pi * (diameter_m / 2) ** 2
            required_velocity = flow_rate_m3s / area

        actual_velocity = self.pipeflow(diameter_m)
        if actual_velocity >= required_velocity:
            adequate_pipe_size = nominal_size

        return adequate_pipe_size

In [5]:
dt = Dunetown()
dt.friction_factor(2100)

0.04867858664517315

In [6]:
dt.pipeflow(.1)

1.0802443144054115

In [7]:
dt.pipe_selector()

24